## Note: all challenges have been directly copied from cryptopals.com.

In [1]:
import codecs
from collections import Counter
import string
from Crypto.Cipher import AES
from Crypto import Random
import random as rnd
import time
import hashlib
import requests
import os

# those variables are used later in several problems so I just initialize them here.
secret_key = Random.get_random_bytes(16)
iv_secret = Random.get_random_bytes(16)
secret_key_16_bits = Random.get_random_bytes(2)

Copied from cryptopals.com
## Crypto Challenge Set 1

This is the qualifying set. We picked the exercises in it to ramp developers up gradually into coding cryptography, but also to verify that we were working with people who were ready to write code.

This set is relatively easy. With one exception, most of these exercises should take only a couple minutes. But don't beat yourself up if it takes longer than that. It took Alex two weeks to get through the set!

If you've written any crypto code in the past, you're going to feel like skipping a lot of this. Don't skip them. At least two of them (we won't say which) are important stepping stones to later attacks. 


## 1-Convert hex to base64

The string:
```
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
```
Should produce:
```
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
```
So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

Cryptopals Rule
```
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.
```

In [2]:
## Challenge 1 convert from hex to base64
## This one is pretty straightforward
input = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
raw = codecs.decode(input, "hex")
result = codecs.encode(raw,"base64")
expected_result = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
result.strip().decode('ascii') == expected_result

True


## 2-Fixed XOR

Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965

... should produce:

746865206b696420646f6e277420706c6179



In [3]:
## Challenge 2 FIxed XOR
input1 = codecs.decode("1c0111001f010100061a024b53535009181c","hex")
input2 = codecs.decode("686974207468652062756c6c277320657965","hex")

def xor_string(string1,string2):
    """ Receive two bytes strings and return the xor in them
        The length of the output is the same as the shortest string
    """
    return bytes(a ^ b for a, b in zip(string1, string2))

expected_result = "746865206b696420646f6e277420706c6179"

codecs.encode(xor_string(input1,input2),"hex").strip().decode('ascii') == expected_result

True

## 3-Single-byte XOR cipher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

Achievement Unlocked

You now have our permission to make "ETAOIN SHRDLU" jokes on Twitter.


In [4]:
## Challenge 3 xor single byte
input1 = codecs.decode("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736","hex")

#Here I'm using as an score of a piece of English plaintext not the character frequency but rather the fact
#that the correct english phrase is more likely to have the most amount of words, this isn't valid in all cases,but worked in this one
biggest = 0;
biggest_character = '';
for character in range(0,128):
    output = xor_string(input1,bytes(chr(character)*len(input1),'ascii'))
    if len(output.decode('ascii').split(' '))>biggest:
        biggest = len(output.decode('ascii').split(' '))
        biggest_character = character
        
output = xor_string(input1,bytes(chr(biggest_character)*len(input1),'ascii'))
output

b"Cooking MC's like a pound of bacon"

## 4-Detect single-character XOR

One of the 60-character strings in this file has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [5]:
## Challenge 4 find encoded string in file

lines = open('4.txt', 'r').readlines()

## I'm gonna use the same logic from the 3rd problem and see the line with the most amont of words
longest_line =""
global_longest = 0;

for line in lines:
    line_raw = codecs.decode(line.strip(),'hex')
    biggest = 0;
    biggest_character = 0;
    for character in range(0,128):
        output = xor_string(line_raw,bytes(len(line_raw)*chr(character),'ascii'))
        flag = False;
        for i in output:
            if i>=128: # if output is not in ascii then it's not an english sentence
                flag = True;
        
        if flag:
            continue
        
        if len(output.decode('ascii').split(' '))>biggest: ## compare between characters in the line
            biggest = len(output.decode('ascii').split(' '))
            biggest_character = character;
            
        if biggest>global_longest: ## compare between all lines
            global_longest=biggest;
            longest_line = output;

print(longest_line)

b'Now that the party is jumping\n'


## 5-Implement repeating-key XOR

Here is the opening stanza of an important work of the English language:
```
Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
```
Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272
a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f

Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.


In [6]:
## Challenge 5 repeating key xor

#def repeating_xor(input1,key):
#    output = b''
#    while len(input1)>0:
#        output += xor_string(input1,key) 
#        input1 = input1[len(key):]
#    return output

# this one is a little bit faster, although the other function is easier to read and uses previous functions
def repeating_xor(input1,key):
    output=[]
    for s in range(0,len(input1)):
        output.append(input1[s] ^ key[s % len(key)])
    return bytes(output)


input1 = "Burning 'em, if you ain't quick and nimble"
key = "ICE"
input2 = "I go crazy when I hear a cymbal"

input3 = input1+ '\n' +input2;
result1 = repeating_xor(bytes(input1,'ascii'),bytes(key,'ascii'))
result2 = repeating_xor(bytes(input2,'ascii'),bytes(key,'ascii'))
result3 = repeating_xor(bytes(input3,'ascii'),bytes(key,'ascii'))

expected1 = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272"
expected2 = "a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

codecs.encode(result3,'hex').decode('ascii') == expected1 + expected2

True

## 6-Break repeating-key XOR
```
It is officially on, now.

This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.
```

There's a file here. It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
```
    this is a test
```
    and
```
    wokka wokka!!!
```
is 37. Make sure your code agrees before you proceed.
3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
7. Solve each block as if it was single-character XOR. You already have code to do this.
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.
```
No, that's not a mistake.

We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.
```

In [7]:
## Challenge 6. Break a repeating xor key( aka Vigenere cypher)
## this is the most challeging exercise so far, but they broke it down step by step so lets follow them

# 2 - Write a function to compute the edit distance/Hamming distance between two strings
# There are probably faster applications of this function in wikipedia
def hamming_distance(bytes1,bytes2):
    after_xor = xor_string(bytes1,bytes2); #xor so that the different bits will became '1'
    return bin(int.from_bytes(after_xor,'little')).count('1') #count the number of '1' bits on the resulting string

#test is the hamming function works as intended;
input1 = "this is a test"
input2 = "wokka wokka!!!"
assert hamming_distance(bytes(input1,'ascii'),bytes(input2,'ascii')) == 37

raw_file = open('6.txt','rb').read()
raw_file_decoded = codecs.decode(raw_file, "base64")

#3- For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit 
#distance between them. Normalize this result by dividing by KEYSIZE.
keysizes = [] #it's gonna be an array of tuples
for keysize in range(2,41):
    distances = []
    pieces = [raw_file_decoded[i:i+keysize] for i in range(0,len(raw_file_decoded),keysize)] #split the file in pieces of size keysize
    
    for i in range(0,len(pieces),2):
        if i+1<len(pieces): #if there are at least two pieces remaining
            distances.append(hamming_distance(pieces[i],pieces[i+1])/keysize) # calculate hamming distance and normalize
    
    distance = sum(distances)/len(distances); #average
    keysizes.append((keysize,distance))
            
            
#4 - The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the
#smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
keysizes_sorted = sorted(keysizes,key=lambda tup: tup[1])
keysizes_selected = [a[0] for a in keysizes_sorted[:4]]

#5- Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
#6 -Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
def GetNthLetters(text, position, keysize,debug=False):
    builtarray = []
    for i in range(position ,len(text),keysize):
        builtarray.append(text[i]);
    return bytes(builtarray)

blocks = [] #blocks is a 3-dimension matrix in which de indices(i,j) point to a byte sequence

for i in range(0,len(keysizes_selected)):
    blocks.append([])
    for j in range(0,keysizes_selected[i]):
        blocks[i].append(GetNthLetters(raw_file_decoded,j,keysizes_selected[i]))
        
#7-Solve each block as if it was single-character XOR. You already have code to do this.

# my idea to find text based on number of words don't work anymore in this case.
# I got this function from https://laconicwolf.com/2018/06/05/cryptopals-challenge-4-detect-single-character-xor-encryption/ for letter frequency
def get_english_score(input_bytes):
    """Compares each input byte to a character frequency 
    chart and returns the score of a message based on the
    relative frequency the characters occur in the English
    language
    """

    # From https://en.wikipedia.org/wiki/Letter_frequency
    # with the exception of ' ', which I estimated.
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000, '@': -50000,
        '^': -50000, '#': -50000, '&': -50000, '~': -50000
    }
    pontuacao = sum([character_frequencies.get(chr(byte), 0) for byte in input_bytes.lower()])
    return pontuacao

def find_letter_xor(input1,byte=False):
    """Find the single byte xor_cypher that is most likely to be correct
        if byte = True, the input is treated as an byte string, otherwise, like ascii text
    """
    highest_score = 0;
    highest_character = 0;
    loop = 128
    if byte:
        loop=256
        
    for character in range(0,loop):
        if byte:
            after_xor = xor_string(input1,len(input1)*int.to_bytes(character,1,'little'))
        else:
            after_xor = xor_string(input1,bytes(len(input1)*chr(character),'ascii'))
        score = get_english_score(after_xor);
        
        if score>highest_score:
            highest_score = score
            highest_character = character
    
    if byte:
        return highest_character
    
    return chr(highest_character)

#8-For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.
keys = {} #array of strings
for i in range(0,len(keysizes_selected)):
    keys[i] = ""
    for j in range(0,keysizes_selected[i]):
        keys[i] += find_letter_xor(blocks[i][j]);
        
keys

{0: 'Terminator X: Bring the noise',
 1: '\x00\x00B\x1e\x04U\x00`H\x00e\x1ee\x1a\x01}v\x07VS\x06\x06\x0b|P',
 2: 'OBuiT\x00e\x00eNq\x00BlesNTOaVs\x00IYfTFRe\x11f`:et\x1aOsi',
 3: 'e\x16lNRrgOn\x07[R{e\x06lOa\x1c\x00\x06RiT\x11\x00\x06U`RnE\x06'}

## 7-AES in ECB mode

The Base64-encoded content in this file has been encrypted via AES-128 in ECB mode under the key
```
"YELLOW SUBMARINE".
```
(case-sensitive, without the quotes; exactly 16 characters; I like "YELLOW SUBMARINE" because it's exactly 16 bytes long, and now you do too).

Decrypt it. You know the key, after all.

Easiest way: use OpenSSL::Cipher and give it AES-128-ECB as the cipher.
```
Do this with code.

You can obviously decrypt this using the OpenSSL command-line tool, but we're having you get ECB working in code for a reason. You'll need it a lot later on, and not just for attacking ECB.
```

In [8]:
# Challenge 7 AES-128 in ecb mode 
# ECB is the mode where each block is encrypted/decrypted separately.
# This one is pretty straightforward, just got to use the AES object from the Crypto.cypher module
key = "YELLOW SUBMARINE"
algo = AES.new(bytes(key,'ascii'),AES.MODE_ECB)

raw_file = codecs.decode(open('7.txt','rb').read(),"base64")
print(algo.decrypt(raw_file).decode('ascii'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## 8-Detect AES in ECB mode

In this file are a bunch of hex-encoded ciphertexts.

One of them has been encrypted with ECB.

Detect it.

Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.


In [9]:
## Challenge 8 Detect AES-128 ecb in file
# This is pretty simple, given the property of ECB given above, it's safe to assume that in an long enough
# series of data an ECB encryption will eventually repeat some blocks. If we count the number of repeted blocks
# we can guess which cyphertext used ECB

def detect_repetitions(cyphertext,keysize):
    number_of_repetitions = 0
    for a in range(0,keysize):
        pieces = [cyphertext[(i+a):(i+keysize+a)] for i in range(0,len(cyphertext),keysize)]
        number_of_repetitions += (len(pieces) - len(set(pieces)))
    return number_of_repetitions

raw_file = open('8.txt','rb').readlines()
cyphertexts = [codecs.decode(cyphertext.strip(),'hex') for cyphertext in raw_file]

repetitions = []
for a in range(0,len(cyphertexts)):
    number_of_repetitions = detect_repetitions(cyphertexts[a],16)
    repetitions.append((a,number_of_repetitions))
    
repetitions_sorted = sorted(repetitions,key=lambda tup: tup[1],reverse=True)
# They haven't given a way to verify which one is the actual answer, but only one cyphertext
# had repetitions at all.
repetitions_sorted[:2]

[(132, 3), (0, 0)]

# Crypto Challenge Set 2

This is the first of several sets on block cipher cryptography. This is bread-and-butter crypto, the kind you'll see implemented in most web software that does crypto.

This set is relatively easy. People that clear set 1 tend to clear set 2 somewhat quickly.

Three of the challenges in this set are extremely valuable in breaking real-world crypto; one allows you to decrypt messages encrypted in the default mode of AES, and the other two allow you to rewrite messages encrypted in the most popular modes of AES. 

## 9-Implement PKCS#7 padding

A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.

One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.

So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,
```
"YELLOW SUBMARINE"
```
... padded to 20 bytes would be:
```
"YELLOW SUBMARINE\x04\x04\x04\x04"
```


In [10]:
## Challenge 9 Implement PKCS#7 padding

def pkcs_padding(text,size):
    padding_size = size-len(text);
    if padding_size<0:
        raise ValueError('Block smaller than text')
    padding = padding_size*chr(padding_size);
    return text+bytes(padding,'ascii')

def pkcs_padding_long(text,size):
    """Apply pkcs padding to an string of any length
       Size is the blocksize.
    """
    pieces = [text[i:i+size] for i in range(0,len(text),size)]
    pieces[-1] = pkcs_padding(pieces[-1],size)
    return b''.join(pieces)
    
def pkcs_unpadding(text,size):
    padding_size = int(text[-1]);
    padding = text[(size-padding_size):];
    if len(set(padding))==1:
        return text[:(size-padding_size)];
    else:
        return text
        
def pkcs_unpadding_long(text,size):
    """Remove pkcs padding from an string of any length
        Size is the blocksize.
    """
    pieces = [text[i:i+size] for i in range(0,len(text),size)]
    pieces[-1] = pkcs_unpadding(pieces[-1],size)
    return b''.join(pieces)
    
input1 = "YELLOW SUBMARINE"
output1 = pkcs_padding_long(bytes(input1,'ascii'),20)
print(output1)
pkcs_unpadding_long(output1,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'


b'YELLOW SUBMARINE'

## 10 - Implement CBC mode

CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)


>***Don't cheat***.
>
>***Do not use OpenSSL's CBC code to do CBC mode, even to verify your results***. What's the point of even doing this stuff if you aren't going to learn from it?


Note: This block here is not a part of the question, it's just an comment of mine.

Here are some diagrams that I found useful when making the code. They're from Wikipedia

![](cbc_encrypt.png)
![](cbc_decrypt.jpg)

In [11]:
## Challenge 10 Implement CBC mode

def cbc_encrypt_manual(cleartext,key,iv):
    pieces = [cleartext[i:i+len(key)] for i in range(0,len(cleartext),len(key))]
    pieces[-1] = pkcs_padding(pieces[-1],len(key));
    algo = AES.new(key,AES.MODE_ECB);

    previous = iv;
    cyphertext = b'';
    for i in range(0,len(pieces)):
        input1 = xor_string(pieces[i],previous);
        output1 = algo.encrypt(input1);
        previous = output1;
        cyphertext += output1
        
    return cyphertext

def cbc_decrypt_manual(cyphertext,key,iv,padding=True):
    pieces = [cyphertext[i:i+len(key)] for i in range(0,len(cyphertext),len(key))]
    pieces[-1] = pkcs_padding(pieces[-1],len(key));
    algoritmo = AES.new(key,AES.MODE_ECB);

    previous = iv;
    cleartext = b'';
    for i in range(0,len(pieces)):
        block_output = algoritmo.decrypt(pieces[i]);
        output = xor_string(block_output,previous);
        previous = pieces[i];
        cleartext += output
    
    if padding:
        return pkcs_unpadding_long(cleartext,len(key))
    else:
        return cleartext
        
raw_file = open('10.txt','rb').read()
raw_file = codecs.decode(raw_file,'base64')
key = b"YELLOW SUBMARINE"
iv = b'\x00'*len(key)

result = cbc_decrypt_manual(raw_file,key,iv)
print(result)

#this is just to test if my encryption function also works. 
test = b"Dancin', Dancin', Dancin'\nCome on, come on, come on everybody\nCome on, come on everybody\nCome on everybody, come on, come on everybody\nCome on, come on everybody\nLet's do this dance"
cbc_decrypt_manual(cbc_encrypt_manual(test,key,iv),key,iv)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

b"Dancin', Dancin', Dancin'\nCome on, come on, come on everybody\nCome on, come on everybody\nCome on everybody, come on, come on everybody\nCome on, come on everybody\nLet's do this dance"

## 11- An ECB/CBC detection oracle

Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:
```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.


In [14]:
## Challenge 11 An ECB/CBC detection oracle

# Random is from the Crypto package and generates random bytes that are cryptographically secure
# Alternatively, one can use os.urandom() to generate random bytes with the same standars, but 
# since I already imported module Crypto to use AES, I'm gonna use it
# rnd is the python random module. Although not cryptographically secure, it's good for simulations
def encryption_oracle(cleartext):
    key = Random.get_random_bytes(16);
    salt = Random.get_random_bytes(rnd.randint(5,10));
    pepper = Random.get_random_bytes(rnd.randint(5,10));
    iv = Random.get_random_bytes(16);
    
    cleartext = salt + cleartext + pepper;
    mode = ''
    if rnd.randint(1,2) == 2:
        cyphertext = cbc_encrypt_manual(cleartext,key,iv);
        mode = 'cbc';
    else:
        algoritmo = AES.new(key,AES.MODE_ECB);
        cyphertext = algoritmo.encrypt(pkcs_padding_long(cleartext,16));
        mode = 'ecb';
    #the oracle return both the cyphertext and the mode cause I can then verify it later
    return [cyphertext,mode]

def detect_ecb_or_cbc(cyphertext,keysize):
    #this is from the challenge 8
    if detect_repetitions(cyphertext,keysize)>0:
        return 'ecb'
    else:
        return 'cbc'

#test to see if works as intented. Since the text needs to be long I picked an not so random wikipedia article
text = open('11.txt','rb').read()
sucesses = 0;
failures = 0;
#I disabled it cause it takes a little time to run and I don't need to run this every time I open the notebook
enabled = False
if enabled:
    for i in range(0,100):
        output = encryption_oracle(text)
        repetitions = detect_repetitions(output[0],16);
        guess = detect_ecb_or_cbc(output[0],16);
        if output[1] == guess:
            sucesses+=1;
        else:
            failures+=1;
        
print([sucesses,failures])

[100, 0]
